In [1]:
pip install chromadb langchain-chroma langchain langchain-community langchain-text-splitters langchain-groq unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
from transformers import pipeline

# Langchain modules
from langchain.document_loaders import UnstructuredFileLoader, PyPDFDirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TokenTextSplitter,
)
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer

# Set API Key for Groq, instead of OpenAI
os.environ["GROQ_API_KEY"] = "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"


# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2')

<ipython-input-2-bc09efe7c1e0>:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

First of all we have to convert the file to txt so that it becomes easy for transfering data into vector database. 4 hours of process takes 15-20 minutes or even less.

In [3]:
# This is to convert .csv file to .txt file
# Read the CSV file
file_path = "/content/services_data_2001.csv"
csv_data = pd.read_csv(file_path)

# You can choose a specific column, or iterate through all rows
text_chunks = []
for index, row in csv_data.iterrows():
    # Combine all columns into one string for each row (adjust as necessary)
    text_row = " ".join(str(value) for value in row.values)
    text_chunks.append(text_row)

# Join all text chunks into a single string
full_text = "\n".join(text_chunks)

# Write the text to a .txt file
with open("service_data_2001.txt", "w") as txt_file:
    txt_file.write(full_text)

print("The text has been successfully written to data_output.txt")

The text has been successfully written to data_output.txt


Loading the documents (txt)- to convert into vector database

In [4]:
file_path = "/content/service_data_2001.txt"
loader = TextLoader(file_path)
documents = loader.load()

In [5]:
persist_directory = "service_data_2001"

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20) - This is method 1
text_chunks = text_splitter.split_documents(documents)
print(f"Number of chunks: {len(text_chunks)}")

Number of chunks: 1786


In [7]:
vectordb = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [9]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)
# retriever
retriever = vectordb.as_retriever()
# Download Sentence Transformers Embedding From Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
# LLM from Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Configuration for conversation buffer memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="result")

# Create the QA chain with memory
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    memory=memory  # Pass the memory here
)

# Interactive loop for querying
print("Welcome to the chatbot. Type 'exit' to stop.")
while True:
    query = input("You: ")

    if query.lower() == 'exit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Get the response from the QA chain
    response = qa_chain.invoke({"query": query})

    # Print the result
    print("Bot: ", response["result"])

    # Optionally print the source document (first result)
    if response.get("source_documents"):
        print("Source document: ", response["source_documents"][0].metadata["source"])

    # Debug: Show current conversation memory (optional)
    print("\nMemory so far:\n", memory.buffer)

Welcome to the chatbot. Type 'exit' to stop.
You: What are the labor hours for fixing Spark Plugs on ford explorer 2000
Bot:  The labor hours for spark plug replacement on a 2000 Ford Explorer are as follows:

- 2.4 Automatic SUV: 1.44 hours
- 3.2 Automatic Pickup Trucks: 1.56 hours
- 1.8 Manual Pickup Trucks: 1.56 hours
- 5.7 Automatic Pickup Trucks: 1.56 hours

Please note that these labor hours may vary depending on the specific circumstances and the mechanic's expertise. These values are based on the provided context.
Source document:  /content/service_data_2001.txt

Memory so far:
 [HumanMessage(content='What are the labor hours for fixing Spark Plugs on ford explorer 2000', additional_kwargs={}, response_metadata={}), AIMessage(content="The labor hours for spark plug replacement on a 2000 Ford Explorer are as follows:\n\n- 2.4 Automatic SUV: 1.44 hours\n- 3.2 Automatic Pickup Trucks: 1.56 hours\n- 1.8 Manual Pickup Trucks: 1.56 hours\n- 5.7 Automatic Pickup Trucks: 1.56 hours\n\n

Practice